<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/lease_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.4 MB/s 
     |████████████████████████████████| 62 kB 796 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def lease_findifNull(research, lo_left,lo_right,la_left,la_right, zip, bedroom):
  result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {}),(BedroomsTotal = {}), (BathroomsTotalInteger ={}+),(BathroomsTotalInteger ={}-)'.format(lo_left, lo_right, la_left, la_right, zip, bedroom, bedroom-2, bedroom + 1))
  lis = []
  count = 0
  di ={}
  df = pd.DataFrame()
  for item in result:
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
            di = item # 存表头
  for key in di:
    lis.append(key)
  if(count >= 5): 
    df = df.T
    df.columns = lis
    return df
  else:
    lo_left -= 0.008
    lo_right += 0.008
    la_left -= 0.008
    la_right += 0.008

    return lease_findifNull(research, lo_left,lo_right,la_left,la_right,zip,bedroom)

In [ ]:
def lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,zip,column,bedroom, a):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(BathroomsTotalInteger = {}+), (BathroomsTotalInteger = {}-), (MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {}), (BedroomsTotal = {})'.format(bedroom - 2, bedroom + 1, house_leftLo, house_rightLo, house_leftLa, house_rightLa, zip, bedroom))
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1         
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008
    a += 1
    if(a >= 12):
      house_leftLo += 0.096
      house_rightLo -= 0.096
      house_leftLa += 0.096
      house_rightLa -= 0.096  #
      #for item in result: 
        #if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            #df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
      #df = df.T
      #df.columns = column #
      return nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column,bedroom)
      
    return lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,zip,column,bedroom, a)

In [ ]:
def nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column,bedroom):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(BathroomsTotalInteger = {}+), (BathroomsTotalInteger = {}-), (MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (BedroomsTotal = {})'.format(bedroom - 2, bedroom + 1, house_leftLo, house_rightLo, house_leftLa, house_rightLa, bedroom))
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1         
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008
    return nullZiphouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, research, column, bedroom)

In [ ]:
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    

In [ ]:
def insertId(pulled_value, mls,rets_client):
  column = []
  for key in pulled_value:
    column.append(key)
  print("\nThis value exists in Website")
  zip = pulled_value.get('PostalCode')
  latitude = pulled_value.get('Latitude')
  longitude = pulled_value.get('Longitude')
  bedroom = pulled_value.get('BedroomsTotal')
  if(zip != "" and latitude != "" and longitude != "" and bedroom != ""):
    if(int(bedroom) >= 3 and int(bedroom) <= 5):
      house_leftLa = float(pulled_value.get('Latitude'))
      house_rightLa = float(pulled_value.get('Latitude')) 
      house_leftLo = float(pulled_value.get('Longitude'))
      house_rightLo = float(pulled_value.get('Longitude'))
      result1 = lease_findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, rets_client,int(zip) ,column,int(bedroom), 0)        
      result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
      price = []
      for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
      result1['ClosePrice'] = price
      mean1 = result1['ClosePrice'].mean()
      top1 = list(result1['ClosePrice'])
      top1.sort(reverse = True)
      top1_mean = sum(top1[0:3])/3
      print('The mean for {} is {}'.format(mls, mean1))
      print('The top 3 for {} are {}'.format(mls, top1[0:3]))
      print('The average for top 3 {} is {}'.format(mls, top1_mean))
      result1.to_csv('{}.csv'.format(mls))
      return
    else:
      print("Bedroom number is not qualified")
      return

In [ ]:
def findAdd(rets_client, address, city, state, zipcode, bedroom):  
  result = cg.address(address, city = city, state= state,zipcode = zipcode)
  lo_left = result[0]['coordinates']['x']
  lo_right = result[0]['coordinates']['x']
  la_left = result[0]['coordinates']['y']
  la_right = result[0]['coordinates']['y']
  result = lease_findifNull(rets_client, lo_left, lo_right, la_left, la_right, int(zipcode), bedroom)
  
  result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
  price = []
  for item in result['ClosePrice']:
    item = float(item)
    price.append(item)
  result['ClosePrice'] = price
  mean = result['ClosePrice'].mean()
  top = list(result['ClosePrice'])
  top.sort(reverse = True)
  print('The mean for {}, {}, {} is {}'.format(address, state, zip, mean))
  print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top[0:3]))
  return result.to_csv('{},{},{}.csv'.format(address, state, zip))

In [ ]:
def get(i,finalN,number,code,rets_client,street,city,state):
  pulled_value = {}
  result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))
  for item in result:
    pulled_value = item
  if(len(pulled_value) != 0):
    column = []
    for key in pulled_value:
      column.append(key)
    print("\nThis value exists in Website")
    zip = pulled_value.get('PostalCode')
    latitude = pulled_value.get('Latitude')
    longitude = pulled_value.get('Longitude')
    bedroom = pulled_value.get('BedroomsTotal')
    if(zip != "" and latitude != "" and longitude != "" and bedroom != ""):
      if(int(bedroom) >= 3 and int(bedroom) <= 5):
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        zip = int(pulled_value.get('PostalCode'))
        result1 = lease_findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, rets_client,zip,column,int(bedroom), 0)        
        result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result1['ClosePrice']:
          item = float(item)
          price.append(item)
        result1['ClosePrice'] = price
        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        top1_mean = sum(top1[0:3])/3
        print('The mean for {}, {}, {} is {}'.format(number[i],finalN[i],code[i], mean1))
        print('The top 3 for {}, {}, {} are {}'.format(number[i],finalN[i],code[i], top1[0:3]))
        print('The average for top 3 {}, {}, {} is {}'.format(number[i],finalN[i],code[i], top1_mean))
        result1.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i]))
        return
      else:
        print("Bedroom number is not qualified")
        return
    else:
      print('check coordinates')
      bedroom = 3
      findAdd(rets_client, street[i], city[i], state[i], code[i], bedroom)
      return 
  else:
    print('check coordinates')
    bedroom = 3
    findAdd(rets_client, street[i], city[i], state[i], code[i], bedroom)
    return 


In [ ]:
def judge(i):
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'

In [ ]:
def main():
  login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
  username = '0671181_NID'
  password = 'Rt$tg6jx'
  rets_client = Session(login_url, username, password)
  rets_client.login()
  while True:
        print('You want to insert values or a csv, please either insert csv or value!')
        choice = input()
        if choice == 'csv':
            while True:
                print('select which csv you want to use')
                csv = input()
                try:
                    data = pd.read_csv('{}.csv'.format(csv))        
                    code = []
                    name = []
                    number = []
                    street = []
                    city = []
                    state = []
                    for item in data['地址']:
                        a = item.split(',')[0]
                        b = item.split(',')[2]
                        c = item.split(',')[1]
                        street.append(a)
                        city.append(c)
                        numbers = a.split()[0]
                        names = a.split()[1:(len(a.split()) - 1)]
                        codes = b.split()[1][0:5]
                        states = b.split()[0]
                        state.append(states)
                        code.append(codes)
                        name.append(names)
                        number.append(numbers)
                    finalN = []
                    for item in name:
                        string = ""
                        for items in item:
                            string += items
                            string += ' '
                        finalN.append(string[0:len(string) - 1])            
                    for i in range(len(code)):
                        get(i,finalN,number,code,rets_client,street,city,state)
                        
                    return "finished"
                except FileNotFoundError:
                    print("Oops! There is no file!  Try again...")
        elif(choice == 'value'):
            while True:
                print('How many mls you want to check?')
                number = input()
                try:
                    Id_list = []
                    for i in range(int(number)):
                        print('Enter the {}{} #MLS'.format(i+1, judge(i+1)))
                        mls = input()
                        Id_list.append(mls)
           #14730621
                    for i in range(len(Id_list)):
                        pulled_value = {}
                        result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(Id_list[i]))
                        for items in result:
                            pulled_value = items
                        if(len(pulled_value) != 0):
                            insertId(pulled_value, Id_list[i], rets_client) 
                            if(i == int(number) - 1):
                                return
                        else:

                            print('Id {} not exists, please insert specific value'.format(Id_list[i]))
                            print("If you don't know the address, please insert skip")
                            deci = input()
                            if(deci == 'skip'):
                                if(i == int(number) - 1):
                                    return
                                continue
                                
                            else:
                                print('Enter address')
                                address = input()
        #'8615 Barclay Street'
                            
                                print('Enter city')
                                city = input()
        #'Dallas'
                                print('Enter state')
                                state = input()
        #'Texas'
                                print('Enter zipcode')
                                zip = input()
        #'75227'
                                print('How many bedrooms?')
                                bedroom = input()

                                try:
                                  findAdd(rets_client, address, city, state, zip, int(bedroom))
                                  if(i == int(number) - 1):
                                      return
                                except IndexError:
                                    print("invalid input, please next value!")   

                          
                except ValueError:
                    print("invalid input, try again!")
        else:
            print("invalid input, try again!")

In [ ]:
if __name__ == "__main__":
    print(main()) 
    #5005 Bama Drive, Arlington, Texas 76017

You want to insert values or a csv, please either insert csv or value!
csv
select which csv you want to use
test1

This value exists in Website
check coordinates
The mean for 103 Robin Ln, TX, <class 'zip'> is 2126.6666666666665
The top 3 for 103 Robin Ln, TX, <class 'zip'> are [2495.0, 2350.0, 2350.0]

This value exists in Website
The mean for 926, Lemontree, 76017 is 1901.4285714285713
The top 3 for 926, Lemontree, 76017 are [2000.0, 2000.0, 1960.0]
The average for top 3 926, Lemontree, 76017 is 1986.6666666666667

This value exists in Website
The mean for 3808, Summer Hill, 75180 is 1735.8333333333333
The top 3 for 3808, Summer Hill, 75180 are [1950.0, 1800.0, 1795.0]
The average for top 3 3808, Summer Hill, 75180 is 1848.3333333333333

This value exists in Website
The mean for 4006, Rolling Rock, 75126 is 2199.0
The top 3 for 4006, Rolling Rock, 75126 are [2553.0, 2400.0, 2275.0]
The average for top 3 4006, Rolling Rock, 75126 is 2409.3333333333335

This value exists in Website
The

In [ ]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()


result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(BathroomsTotalInteger = {}+), (BathroomsTotalInteger = {}-), (MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {}), (BedroomsTotal = {})'.format(0, 100, -98.71837, -94.71837, 30.8928, 34.8928, 75238, 3))
for item in result:
  if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
    print(item)

In [ ]:
df = pd.read_csv('test.csv')
df.rename(columns={ df.columns[0]: "地址" }, inplace = True)
df.to_csv('test1.csv')